In [1]:
import torch
import torchvision

In [2]:
import gym


In [3]:
env = gym.make('CartPole-v0')
env.reset()
for _ in range(10):
    env.render()
    env.step(env.action_space.sample()) # take a random action

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [4]:
import random
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import gym
from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
)

#env = gym.make('FrozenLake8x8-v0')
#env = gym.make('FrozenLake-v0')
env = gym.make('FrozenLakeNotSlippery-v0')
env.render()
class pi_net(nn.Module):
    def __init__(self):
        super(pi_net, self).__init__()
        self.linear1 = nn.Linear(1, 20)
        self.linear2 = nn.Linear(20, 4)

    def forward(self, x):
        #print(x.shape)
        #print(x)
        x = F.relu(self.linear1(x))
        #x = F.softmax(self.linear2(x), dim=0)
        x = self.linear2(x)
        return x.view(-1, 4)
    





SFFF
FHFH
FFFH
HFFG


In [6]:
# custom weights initialization 
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        #m.weight.data.normal_(0.0, 0.02)
        #m.weight.data.uniform_(0.0, 0.02)
        m.weight.data.fill_(0.5)

import gym
import numpy as np
import torch.optim as optim
from torch.distributions import Categorical

NUM_EPISODES = 1000000
GAMMA = 0.9
net = pi_net()
print "a"
net.apply(weights_init)
print "b"
optimizer = optim.RMSprop(net.parameters(), lr=0.0001)


score = []
times_trained = 0
times_reach_goal = 0
for k in range(NUM_EPISODES):
    done = False
    observation = env.reset()
    #observation, reward, done, info = env.step(env.action_space.sample()) # take a random action

    episode_series = []
    while not done:
        # Get action from pi
        # action = env.action_space.sample()
        np_observation = np.array(observation)
        #np_observation = np.expand_dims(np_observation, axis=0)
        np_observation = np.expand_dims(np_observation, axis=0)
        observation_tensor = torch.FloatTensor(np_observation) 
        #print(observation_tensor)
        #net.eval()
        #print("before eval")
        action_probs = net(observation_tensor)
        action_probs_orig = action_probs
        #print("action_probs after net")
        #print(action_probs)
        #FOR EXPLORATION: 
        action_probs = F.dropout(action_probs, p=0.3, training=True)
        #print("action_probs after dropout")
        #print(action_probs)
        action_probs = F.softmax(action_probs, dim=1)
        #print("action_probs after softmax")
        #print(action_probs)
        #action = action_probs.multinomial(num_samples=1)
        m = Categorical(action_probs)
        action = m.sample()
        
        
        
        #print("after eval")
        #print("action_probs")
        #print(action_probs)
        log_prob = m.log_prob(action)
        #print("log_prob")
        #print(log_prob)
        #break
        #print("softmax")
        #print(action_probs)
        #print("action")
        #print(str(action.item()))
        #print(type(prob.multinomial))
        
        #break
        # Execute action in environment.
        
        if k%10000 == 0:
            print("action_probs_orig ")
            print(action_probs_orig)
            print("On state="+ str(observation) + ", selected action=" + str(action.item()) + " , ")
        
        observation, reward, done, info = env.step(action.item()) 
        
        if k%10000 == 0:
            print("new state="+ str(observation) + ", done="+str(done))
#         if done and reward != 1.0:
#             reward = -1.0
        step_data = [observation, action,log_prob, reward, done, info]
        episode_series.append(step_data)
        #env.render()
        
   
    if len(score) < 100:
        score.append(reward)
    else:
        score[k % 100] = reward

    if k%1000 == 0:
        print("Episode {} finished after {} timesteps with r={}. Running score: {}. Times trained: {}. Times reached goal: {}.".format(k, len(episode_series), reward, np.mean(score), times_trained, times_reach_goal))
        times_trained = 0
        times_reach_goal = 0
        #print("Game finished. " + "-" * 5)
        #print(len(episode_series))
#         for param in net.parameters():
#             print(param.data)
        
    #break
    #Training:
    #episode_series.reverse()
    policy_loss = []
    rewards_list = []
    for i in range(len(episode_series)):
        j = i
        G = 0
        alpha = 1 / len(episode_series)
        
        # get the log_prob of the last state:
        gamma_cum = GAMMA
        
        while j < len(episode_series): 
            [observation, action, log_prob, reward, done, info] = episode_series[j]
            G = G + reward * gamma_cum
            
            gamma_cum = gamma_cum * GAMMA
            j = j + 1
        policy_loss.append(G * -log_prob)
        rewards_list.append(G)
    
    if G > 0.0 or True: # Optimize only if rewards are non zero.
        #print "Reward list"
        #print rewards_list
        optimizer.zero_grad()
        policy_loss = torch.cat(policy_loss).sum()
        policy_loss.backward()
        optimizer.step()
        times_trained = times_trained + 1
    
    if G > 0.0:
        times_reach_goal = times_reach_goal + 1
        
    
    



a
b
action_probs_orig 
tensor([[ 1.8978,  2.1219,  2.2214,  2.1729]])
On state=0, selected action=0 , 
new state=0, done=False
action_probs_orig 
tensor([[ 1.8978,  2.1219,  2.2214,  2.1729]])
On state=0, selected action=0 , 
new state=0, done=False
action_probs_orig 
tensor([[ 1.8978,  2.1219,  2.2214,  2.1729]])
On state=0, selected action=0 , 
new state=0, done=False
action_probs_orig 
tensor([[ 1.8978,  2.1219,  2.2214,  2.1729]])
On state=0, selected action=1 , 
new state=4, done=False
action_probs_orig 
tensor([[ 19.0885,  19.3126,  19.4121,  19.3636]])
On state=4, selected action=0 , 
new state=4, done=False
action_probs_orig 
tensor([[ 19.0885,  19.3126,  19.4121,  19.3636]])
On state=4, selected action=1 , 
new state=8, done=False
action_probs_orig 
tensor([[ 39.0885,  39.3126,  39.4121,  39.3636]])
On state=8, selected action=1 , 
new state=12, done=True
Episode 0 finished after 7 timesteps with r=0.0. Running score: 0.0. Times trained: 0. Times reached goal: 0.


KeyboardInterrupt: 

In [55]:
a = np.array([1,2,3,4])

In [25]:
a


array([1, 2, 3, 4])

In [26]:
print(a)

[1 2 3 4]


In [27]:
a[::-1]

array([4, 3, 2, 1])